In [2]:
#pip3 install selenium
#pip3 install pandas
#pip3 install requests
#pip install -v python-mecab-ko==1.0.9
#https://buttercoco.tistory.com/26

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pyperclip
import requests
import time
import sys, os
import re
import numpy as np
import warnings
import datetime

keys = Keys()
options = Options()
options.add_argument('incognito')
warnings.filterwarnings(action='ignore') 

In [46]:
def open_chrome():
    #네이버 로그인 화면 이동 
    url = "https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com"
    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    sleep(10)
    
    return driver

def move_site(driver,cafe_url):
    driver = driver 
    driver.get(cafe_url)
    sleep(5)
    
    return driver  

def move_board(driver, number):
    driver = driver 
    css_selector = f"#menuLink{number}"
    driver.find_element(By.CSS_SELECTOR,css_selector).click()
    sleep(5)
    
    return driver

def article_list_n(driver, number):
    driver = driver
    number = number
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.CSS_SELECTOR,"#listSizeSelectDiv > a").click()
    driver.find_element(By.CSS_SELECTOR,f"#listSizeSelectDiv > ul > li:nth-child({number}) > a").click()
    driver.switch_to.default_content()
    sleep(5)
    
    return driver

def article_n(driver):
    driver.switch_to.frame("cafe_main")
    article_n = len(driver.find_elements(By.CLASS_NAME,"inner_number"))
    driver.switch_to.default_content()
    
    return article_n

def move_board_page(driver, article_n):
    n = article_n
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.XPATH,f"/html/body/div[1]/div/div[6]/a[{n}]").click()
    driver.switch_to.default_content()
    sleep(2)    
    return driver

def check_article():
    driver.switch_to.frame("cafe_main")
    n_article = driver.find_elements(By.CLASS_NAME,'nodata')
    driver.switch_to.default_content()
    
    return len(n_article)

def article_n():
    driver.switch_to.frame("cafe_main")
    article_n = len(driver.find_elements(By.CLASS_NAME,"inner_number"))
    driver.switch_to.default_content()
    
    return article_n

def get_month_end(month):
    if month == 2:
        return '28'
    elif month in [4, 6, 9, 11]:
        return '30'
    else:
        return '31'
    
def get_month_end_date(year, month):
    end_day = get_month_end(month)
    if month == 2 and ((year % 4 == 0 and year % 100 != 0) or year % 400 == 0):
        end_day = '29'
    return "{0}{1:02d}{2}".format(year, month, end_day)

def get_month_start(year, month):
    return '{:04d}{:02d}01'.format(year, month)

def check_date(driver, date_start, date_end, list_,article_n):
    driver = driver
    i = list_
    s = int(date_start)
    e = int(date_end)
    n = article_n
    
    driver.switch_to.frame("cafe_main")
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    driver.switch_to.default_content() 
    
    for j in range(0,n):
        i.append(int(soup.select('td.td_date')[j].text.strip().replace(":","").replace(".","")))
    for j in range(0,n):
        if e >= i[j] >= s :
            i[j] = 1
        else :
            i[j] = 0
            
    return i

def move_article_page(driver, article_n):
    n = article_n
    driver = driver 
    
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.XPATH,f"/html/body/div[1]/div/div[4]/table/tbody/tr[{n}]/td[1]/div[2]/div/a[1]").click()
    driver.switch_to.default_content()
    sleep(2)    
    
    return driver
def reply_extract(driver, cafename, dataframe,게시글,댓글):
    d = dataframe
    s = cafename
    driver = driver
    게시글 = 게시글
    댓글 = 댓글 
    
    driver.switch_to.frame("cafe_main")
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    #게시판
    board = soup.select('a.link_board')[0].text.strip()
    
    #제목
    title = soup.select('h3.title_text')[0].text.strip()
    
    #조회수
    views = soup.select('span.count')[0].text.replace('조회', '')
    
    #작성일
    date = soup.select('span.date')[0].text
    
    #url
    cont_url = driver.find_element(By.XPATH,'//*[@id="spiButton"]').get_attribute('data-url')
    
    #게시글
    try:
        word = soup.find("div", attrs={"class":"se-main-container"}).get_text().replace("\n","").replace(" ","").replace("\u200b","")
    except AttributeError as err:
        word = "블로그 url"
    
    driver.switch_to.default_content()
    
    게시글.append(word)
    for i in range(1,len(soup.select('span.text_comment'))+1):
        df = []
        df.append(s)
        df.append(board)
        df.append(title)
        df.append(cont_url)
        df.append(views)
        df.append(date[:10])
        df.append(word)
        df.append(soup.select('span.text_comment')[i-1].text)
        댓글.append(soup.select('span.text_comment')[i-1].text)
        d = d.append(pd.Series(df,index=d.columns),ignore_index=True)
    return d,게시글,댓글

def back_page(driver):
    driver = driver 
    
    driver.back()
    sleep(2)
    return driver

In [47]:
cafename = "브랜디드" 
cafe_url = "https://cafe.naver.com/coredenim"
board_number = 145
article_by_page = 7
year = 2022
month = 8

date_end = get_month_end_date(year, month)
date_start = get_month_start(year, month)
while_date_end = get_month_end_date(year, month-1)
while_date_start = get_month_end_date(year, month-1)

In [48]:
d = pd.DataFrame(columns=["카페명","게시판","게시물","url","조회수","작성날짜","게시글","댓글"])
d3 = pd.DataFrame(columns=["카페명","게시판","게시물","url","조회수","작성날짜","게시글","댓글"])

게시글 = []
댓글 = []

In [49]:
#driver = open_chrome()
driver = move_site(driver, cafe_url)
driver = move_board(driver, board_number)
driver = article_list_n(driver, article_by_page)

In [50]:
#n은 article 갯수 
# i = 첫번째 페이지 여부
# j = 반복문 
# k = 범위에 포함되는지 여부 리스트
# l m n o  
i = 0
l = 0 
while i < 500 :
    try:
        i+=1
        if i==1:
            for j in range(2,12):
                check_date_boolean=[]
                check_date_while_end=[]
                driver = move_board_page(driver,j)
                
                #게시물 확인
                if check_article() > 0:
                    print("게시물이 확인되지 않습니다.")
                    break
                else:
                    num = article_n()
                
                #작성일 확인
                check_date_boolean = check_date(driver, date_start, date_end, check_date_boolean, num)
                check_date_while_end = check_date(driver,while_date_start,while_date_end,check_date_while_end,num)
                
                if sum(check_date_while_end) > 0 : 
                    break;
                    print("기간 끝")
                    break;
                else:
                    k_num = 0
                    for k in check_date_boolean :
                        k_num+=1
                        if k==1:
                            driver = move_article_page(driver,k_num)
                            d2,게시글,댓글 = reply_extract(driver,cafename,d,게시글,댓글)
                            d3 = d3.append(d2)
                            driver = back_page(driver)
        else:
            for j in range(2,13):
                check_date_boolean=[]
                check_date_while_end=[]
                driver = move_board_page(driver,j)
                
                #게시물 확인
                if check_article() > 0:
                    print("게시물이 확인되지 않습니다.")
                    break
                else:
                    num = article_n()
                
                #작성일 확인
                check_date_boolean = check_date(driver, date_start, date_end, check_date_boolean, num)
                check_date_while_end = check_date(driver,while_date_start,while_date_end,check_date_while_end,num)
                
                if sum(check_date_while_end) > 0 : 
                    break;
                    print("기간 끝")
                    break;
                else:
                    k_num = 0
                    for k in check_date_boolean :
                        k_num+=1
                        if k==1:
                            driver = move_article_page(driver,k_num)
                            d2,게시글,댓글 = reply_extract(driver,cafename,d,게시글,댓글)
                            d3 = d3.append(d2)
                            driver = back_page(driver)
    except:
        pass

In [28]:
#n은 article 갯수 
# i = 첫번째 페이지 여부
# j = 반복문 
# k = 범위에 포함되는지 여부 리스트
# l m n o  
i = 0
l = 0 
while i < 500 :
    try:
        i+=1
        if i==1:
            for j in range(2,12):
                check_date_boolean=[]
                check_date_while_end=[]
                driver = move_board_page(driver,j)
                
                #게시물 확인
                if check_article() > 0:
                    print("게시물이 확인되지 않습니다.")
                    break
                else:
                    num = article_n()
                
                #작성일 확인
                check_date_boolean = check_date(driver, date_start, date_end, check_date_boolean, num)
                check_date_while_end = check_date(driver,while_date_start,while_date_end,check_date_while_end,num)
                
                if sum(check_date_while_end) > 0 : 
                    break;
                    print("기간 끝")
                    break;
                else:
                    k_num = 0
                    for k in check_date_boolean :
                        k_num+=1
                        if k==1:
                            driver = move_article_page(driver,k_num)
                            d2,게시글,댓글 = reply_extract(driver,cafename,d,게시글,댓글)
                            d3 = d3.append(d2)
                            driver = back_page(driver)
        else:
            for j in range(2,13):
                check_date_boolean=[]
                check_date_while_end=[]
                driver = move_board_page(driver,j)
                
                #게시물 확인
                if check_article() > 0:
                    print("게시물이 확인되지 않습니다.")
                    break
                else:
                    num = article_n()
                
                #작성일 확인
                check_date_boolean = check_date(driver, date_start, date_end, check_date_boolean, num)
                check_date_while_end = check_date(driver,while_date_start,while_date_end,check_date_while_end,num)
                
                if sum(check_date_while_end) > 0 : 
                    break;
                    print("기간 끝")
                    break;
                else:
                    k_num = 0
                    for k in check_date_boolean :
                        k_num+=1
                        if k==1:
                            driver = move_article_page(driver,k_num)
                            d2,게시글,댓글 = reply_extract(driver,cafename,d,게시글,댓글)
                            d3 = d3.append(d2)
                            driver = back_page(driver)
    except:
        pass

In [66]:
from konlpy.tag import Twitter 
from collections import Counter

In [67]:
twitter = Twitter() 
morphs = [] 

#for sentence in 게시글: 
#    morphs.append(twitter.pos(sentence)) 

#for sentence in 댓글: 
#    morphs.append(twitter.pos(sentence)) 

#print(morphs)

In [53]:
noun_adj_adv_list=[] 

for sentence in morphs : 
    for word, tag in sentence : 
        if tag in ['Noun'] and ("것" not in word) and ("저" not in word) and ("등" not in word) and ("전" not in word) and ("요" not in word) and ("분" not in word) and ("시" not in word) and ("카" not in word) and ("너" not in word) and ("및" not in word) and ("이" not in word) and ("거" not in word) and ("좀" not in word) and ("제" not in word) and ("후" not in word) and ("비" not in word) and ("내" not in word)and ("나" not in word)and ("수"not in word) and("게"not in word)and("말"not in word): 
            noun_adj_adv_list.append(word) 
            
print(noun_adj_adv_list)

['상의', '앤더슨', '하의', '유니클로', '상의', '애프터', '하의', '유니클로', '신발', '앤', '식스', '가방', '애드', '오프', '협찬', '상의', '브라운', '야드', '하의', '리', '신발', '페몬트상', '스', '유니클로', '가방', '애드', '오프', '협찬', '신발', '머렐', '상의', '앤더슨', '하의', '가방', '애드', '오프', '협찬', '상의', '블러', '체험', '단', '하의', '체험', '단', '상의', '블러', '하의', '체험', '단', '가방', '애드', '오프', '협찬', '상의', '우프', '체험', '단', '하의', '신발', '페몬트', '아우', '터', '니스', '상의', '우프', '체험', '단', '하의', '신발', '앤', '식스', '아우', '터', '포르노', '체험', '단', '하의', '앤더슨', '신발', '페몬트', '환절기', '감기', '포', '끌', '짜임', '런', '레몬', '블루', '라벤더', '모음', '핑크', '또더', '현대', '평일', '낮', '사람', '사진', '스펙', '가디건', '링', '멜란지', '탑필', '가디건', '딥퍼플', '보라색', '진짜', '때', '헤어', '리', '소프트', '가디건', '브릭', '멜란지', '블루', '종', '디', '스트레스', '믹스', '드', '레더', '점퍼', '다크', '브라운', '앞', '리신', '극찬', '유가', '빈티', '느낌', '그냥', '스탠', '집업', '블루', '종', '자켓', '블랙', '재질', '살', '때', '체인', '징', '롭스', '트링', '자켓', '다크', '브라운', '램', '스킨크', '리즈', '블루', '종', '블랙', '레더', '트러', '블랙', '레더', '트러', '블랙', '체인', '징', '롭스', '트롱', '자켓', '블랙', '헤링', '레더', '패

In [54]:
count = Counter(noun_adj_adv_list)

words = dict(count.most_common())

words

{'진짜': 2598,
 '더': 1260,
 '바지': 1045,
 '느낌': 992,
 '오': 986,
 '핏': 896,
 '가격': 870,
 '생각': 867,
 '구매': 866,
 '고민': 797,
 '코트': 733,
 '셔츠': 723,
 '블루': 715,
 '블랙': 702,
 '유': 694,
 '자켓': 639,
 '옷': 635,
 '용': 599,
 '개': 596,
 '때': 584,
 '정도': 580,
 '드': 555,
 '맛': 528,
 '리': 521,
 '뭐': 519,
 '색감': 510,
 '사진': 502,
 '그냥': 487,
 '신발': 474,
 '안': 446,
 '바로': 444,
 '팬츠': 439,
 '듯': 438,
 '착샷': 434,
 '투표': 432,
 '지금': 428,
 '그': 422,
 '번': 412,
 '브랜드': 411,
 '보고': 409,
 '추천': 398,
 '기장': 387,
 '컬러': 384,
 '니트': 382,
 '오늘': 380,
 '터': 366,
 '감': 362,
 '데님': 354,
 '중': 349,
 '어디': 347,
 '존잘': 345,
 '도': 340,
 '가디건': 338,
 '종': 337,
 '여': 334,
 '아우': 327,
 '입': 325,
 '걸': 306,
 '린다': 299,
 '또': 296,
 '굿': 291,
 '한번': 288,
 '체험': 283,
 '네': 283,
 '가을': 274,
 '발매': 274,
 '사람': 273,
 '코디': 273,
 '실물': 272,
 '색': 272,
 '해': 271,
 '브랜디': 265,
 '스펙': 264,
 '왜': 264,
 '발': 262,
 '오우': 260,
 '쿠어': 259,
 '임': 259,
 '글': 246,
 '겨울': 244,
 '구': 244,
 '브라운': 241,
 '조금': 239,
 '도리': 238,
 '예정': 237,
 '일단': 

In [55]:
d3 = d3.reset_index()
d3 = d3.drop(["index"],axis=1)
d3.to_csv('/Users/jisoo.kim/Downloads/navercafe.csv',encoding='utf-8')

In [60]:
d3.to_csv('/Users/jisoo.kim/Downloads/navercafe.csv',encoding='utf-8')

In [58]:
def save_dict_to_file(dic):
    f = open('dict.txt','w')
    f.write(str(dic))
    f.close()

def load_dict_from_file():
    f = open('dict.txt','r')
    data=f.read()
    f.close()
    return eval(data)

In [59]:
save_dict_to_file(words)

In [69]:
twitter.nouns("아이오아이")

['아이오', '아이']

In [74]:
from konlpy.tag import Twitter as OriginalTwitter
from ckonlpy.tag import Twitter 
twitter.add_dictionary('아이오아이', 'Noun')

AttributeError: 'Okt' object has no attribute 'add_dictionary'

In [79]:
from konlpy.tag import *
okt = Okt()
print(okt.nouns('DBMS'))

[]


In [83]:
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.add_dictionary(['문제해결능력','DBMS','아이오아이'], 'Noun')

In [84]:
print(twitter.nouns('아이오아이'))

['아이오아이']


In [ ]:
https://jae-eun-ai.tistory.com/20